# Modelling most important features

## Model Selection

In [1]:
# Importing the libraries
import os
import pandas as pd

## Data Preprocessing

In [2]:
# Load and preprocess the train data 
from pipelines import preprocessing_pipeline

data_file = os.path.join('..', '..', '..', '..', 'data', 'raw', 'train.csv')
train_data = pd.read_csv(data_file, index_col=0, low_memory=False)
train_data = preprocessing_pipeline.fit_transform(train_data)
train_data.head()

,day_phase,bg-1:00,bg-0:55,bg-0:50,bg-0:45,bg-0:40,bg-0:35,bg-0:30,bg-0:25,bg-0:20,...,hr-0:05,hr-0:00,cals-0:30,cals-0:25,cals-0:20,cals-0:15,cals-0:10,cals-0:05,cals-0:00,bg+1:00
id,,,,,,,,,,,,,,,,,,,,,
p01_0,morning,17.3,17.366667,17.433333,17.5,17.433333,17.366667,17.3,16.933333,16.566667,...,90.6,99.6,0.96,0.96,0.96,0.96,0.96,0.96,4.8,13.4
p01_1,morning,17.5,17.433333,17.366667,17.3,16.933333,16.566667,16.2,15.833333,15.466667,...,90.6,99.6,0.96,0.96,0.96,0.96,0.96,0.96,4.8,12.8
p01_2,morning,17.3,16.933333,16.566667,16.2,15.833333,15.466667,15.1,14.866667,14.633333,...,90.6,99.6,0.96,0.96,0.96,0.96,0.96,0.96,4.8,15.5
p01_3,morning,16.2,15.833333,15.466667,15.1,14.866667,14.633333,14.4,14.233333,14.066667,...,90.6,99.6,0.96,0.96,0.96,0.96,0.96,0.96,4.8,14.8
p01_4,morning,15.1,14.866667,14.633333,14.4,14.233333,14.066667,13.9,13.866667,13.833333,...,90.6,99.6,0.96,0.96,0.96,0.96,0.96,0.96,4.8,12.7


## Data Splitting

In [3]:
from sklearn.model_selection import train_test_split

X = train_data.drop(columns=['bg+1:00'])
y = train_data['bg+1:00']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Standardization

In [4]:
from pipelines import standardization_pipeline

X_train = standardization_pipeline.fit_transform(X_train)
X_test = standardization_pipeline.transform(X_test)

X_train.head()

,bg-1:00,bg-0:55,bg-0:50,bg-0:45,bg-0:40,bg-0:35,bg-0:30,bg-0:25,bg-0:20,bg-0:15,...,cals-0:25,cals-0:20,cals-0:15,cals-0:10,cals-0:05,cals-0:00,day_phase_evening,day_phase_morning,day_phase_night,day_phase_noon
id,,,,,,,,,,,,,,,,,,,,,
p12_24040,-0.593126,-0.593911,-0.627250,-0.659985,-0.694234,-0.727411,-0.759862,-0.794134,-0.793841,-0.826287,...,-0.214903,-0.232818,-0.221298,-0.217413,-0.211511,-0.201556,0,0,0,1
p12_13267,-0.159478,-0.193086,-0.226517,-0.259754,-0.326886,-0.293335,-0.259428,-0.293076,-0.292998,-0.359421,...,-0.214903,-0.232818,-0.221298,-0.217413,-0.000161,-0.201556,0,1,0,0
p03_131,2.175554,2.178465,2.077695,2.108275,2.144362,2.144167,2.176018,2.245619,2.211220,2.241694,...,-0.859023,-0.842894,-0.858737,-0.853195,-0.854637,-0.852188,0,0,0,0
p12_20861,-0.426338,-0.560509,-0.727433,-0.693337,-0.660838,-0.627239,-0.593050,-0.593711,-0.526725,-0.559506,...,-0.003227,1.764690,0.963619,2.626939,-0.193358,0.652399,0,0,0,0
p12_24627,-0.526411,-0.560509,-0.527067,-0.493222,-0.393676,-0.226554,-0.259428,-0.326480,-0.326387,-0.326073,...,-0.020109,-0.147948,0.372446,0.842650,0.632593,0.066043,0,0,0,0


## Model Selection with LazyPredict

In [5]:
from notebooks.helpers.LazyPredict import get_lazy_regressor

reg = get_lazy_regressor(exclude=['SVR'])
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

 97%|█████████▋| 37/38 [10:08<00:08,  8.20s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003199 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8637
[LightGBM] [Info] Number of data points in the train set: 141619, number of used features: 38
[LightGBM] [Info] Start training from score 8.276012


100%|██████████| 38/38 [10:09<00:00, 16.03s/it]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,0.64,0.64,1.80,88.65
XGBRegressor,0.60,0.60,1.91,0.66
BaggingRegressor,0.59,0.59,1.91,29.36
LGBMRegressor,0.58,0.58,1.95,1.08
HistGradientBoostingRegressor,0.58,0.58,1.96,1.72
MLPRegressor,0.56,0.56,2.00,36.94
KNeighborsRegressor,0.55,0.55,2.01,3.19
GradientBoostingRegressor,0.54,0.54,2.04,53.77
RidgeCV,0.50,0.50,2.12,0.26


The best models are:

* ExtraTreesRegressor
* XGBRegressor
* BaggingRegressor

For further work I choose:

* ExtraTreesRegressor
* XGBRegressor
* BaggingRegressor